In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 1: Load Pre-trained GoogleNet
# Load the pre-trained GoogleNet model from torchvision.models
googlenet = models.googlenet(pretrained=True)

# Modify the final fully connected layer for a custom number of classes
num_classes = 10  # Example: CIFAR-10 has 10 classes
googlenet.fc = nn.Linear(googlenet.fc.in_features, num_classes)

# Move the model to the selected device
googlenet = googlenet.to(device)

# Print the modified model architecture
print(googlenet)

# Step 2: Prepare the Dataset
# Define transformations for the training and validation datasets
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Randomly crop and resize to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

val_transform = transforms.Compose([
    transforms.Resize(256),  # Resize the image to 256x256
    transforms.CenterCrop(224),  # Crop the center 224x224 portion
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

# Load the dataset (example: CIFAR-10)
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=val_transform)

# Create DataLoader objects for training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Step 3: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.SGD(googlenet.parameters(), lr=0.001, momentum=0.9)  # Stochastic Gradient Descent with momentum

# Step 4: Training Loop
num_epochs = 5  # Number of epochs to train
for epoch in range(num_epochs):
    googlenet.train()  # Set the model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        # Move inputs and labels to the selected device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = googlenet(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item()

    # Print epoch loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Step 5: Validation Loop
googlenet.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation
    for inputs, labels in val_loader:
        # Move inputs and labels to the selected device
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = googlenet(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the predicted class

        # Update total and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print validation accuracy
print(f"Validation Accuracy: {100 * correct / total:.2f}%")

# Step 6: Inference on a Test Image
# Load and preprocess the test image
input_image = Image.open("path_to_image.jpg")  # Replace with the path to your image
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# Move the input to the selected device
input_batch = input_batch.to(device)

# Perform inference
with torch.no_grad():
    output = googlenet(input_batch)

# Get the predicted class
_, predicted_class = torch.max(output, 1)
print(f"Predicted class: {predicted_class.item()}")

c:\Users\DELL\miniconda3\envs\cv_live\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\DELL\miniconda3\envs\cv_live\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to C:\Users\DELL/.cache\torch\hub\checkpoints\googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:03<00:00, 17.2MB/s]


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

100%|██████████| 170M/170M [00:26<00:00, 6.45MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
